In [1]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 transform, col, concat, lit

# 기본 데이터 변환 작업을 위한 SparkSession 생성
spark = (SparkSession.builder  # SparkSession 빌더 패턴 시작
         .appName("basic-transformations")  # 기본 변환 애플리케이션
         .master("spark://spark-master:7077")  # Spark 마스터 연결
         .config("spark.executor.memory", "512m")  # 실행자 메모리 설정
         .getOrCreate())  # 세션 생성 또는 기존 세션 사용

# 로그 레벨을 ERROR로 설정
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/04 16:28:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 노벨상 데이터 JSON 파일 로드
df = (spark.read.format("json")  # JSON 형식 지정
      .option("multiLine", "true")  # 여러 줄 JSON 처리
      .load("../data/nobel_prizes.json"))  # 노벨상 데이터 파일 로드

In [3]:
# transform 함수를 사용하여 배열 데이터 변환 - 수상자 이름 결합
df_transformed = (
    df.select("category"  # 카테고리 선택
              , "overallMotivation"  # 전체 동기 선택
              , "year"  # 연도 선택
              , "laureates"  # 수상자 배열 선택
              , transform(col("laureates"), lambda x: concat(x.firstname,lit(" "), x.surname))  # 배열의 각 요소에 대해 이름+성 결합
              .alias("laureates_full_name")))  # 새 컴럼 이름 지정

# 변환된 DataFrame 출력
df_transformed.show()  # DataFrame 내용 출력

+----------+--------------------+----+--------------------+--------------------+
|  category|   overallMotivation|year|           laureates| laureates_full_name|
+----------+--------------------+----+--------------------+--------------------+
| chemistry|                null|2022|[{Carolyn, 1015, ...|[Carolyn Bertozzi...|
| economics|                null|2022|[{Ben, 1021, "for...|[Ben Bernanke, Do...|
|literature|                null|2022|[{Annie, 1017, "f...|      [Annie Ernaux]|
|     peace|                null|2022|[{Ales, 1018, "Th...|[Ales Bialiatski ...|
|   physics|                null|2022|[{Alain, 1012, "f...|[Alain Aspect, nu...|
|  medicine|                null|2022|[{Svante, 1011, "...|      [Svante Pääbo]|
| chemistry|                null|2021|[{Benjamin, 1002,...|[Benjamin List, D...|
| economics|                null|2021|[{David, 1007, "f...|[David Card, Josh...|
|literature|                null|2021|[{Abdulrazak, 100...| [Abdulrazak Gurnah]|
|     peace|                

In [4]:
# 중복 데이터 제거 - 지정된 컴럼들을 기준으로 중복 행 삭제
df_deduped = df.dropDuplicates(  # 중복 데이터 제거["category","overallMotivation", "year"])

# 중복 제거된 DataFrame 출력
df_deduped.show()  # DataFrame 내용 출력

+---------+--------------------+-----------------+----+
| category|           laureates|overallMotivation|year|
+---------+--------------------+-----------------+----+
|chemistry|[{Jacobus H., 160...|             null|1901|
|chemistry|[{Emil, 161, "in ...|             null|1902|
|chemistry|[{Svante, 162, "i...|             null|1903|
|chemistry|[{Sir William, 16...|             null|1904|
|chemistry|[{Adolf, 164, "in...|             null|1905|
|chemistry|[{Henri, 165, "in...|             null|1906|
|chemistry|[{Eduard, 166, "f...|             null|1907|
|chemistry|[{Ernest, 167, "f...|             null|1908|
|chemistry|[{Wilhelm, 168, "...|             null|1909|
|chemistry|[{Otto, 169, "in ...|             null|1910|
|chemistry|[{Marie, 6, "in r...|             null|1911|
|chemistry|[{Victor, 172, "f...|             null|1912|
|chemistry|[{Alfred, 174, "i...|             null|1913|
|chemistry|[{Theodore W., 17...|             null|1914|
|chemistry|[{Richard, 176, "...|             nul

In [5]:
# 연도를 기준으로 오름차순 정렬
df_sorted = df.orderBy(  # 정렬"year")

# 정렬된 DataFrame 출력
df_sorted.show()  # DataFrame 내용 출력

+----------+--------------------+-----------------+----+
|  category|           laureates|overallMotivation|year|
+----------+--------------------+-----------------+----+
| chemistry|[{Jacobus H., 160...|             null|1901|
|literature|[{Sully, 569, "in...|             null|1901|
|     peace|[{Henry, 462, "fo...|             null|1901|
|   physics|[{Wilhelm Conrad,...|             null|1901|
|  medicine|[{Emil, 293, "for...|             null|1901|
| chemistry|[{Emil, 161, "in ...|             null|1902|
|literature|[{Theodor, 571, "...|             null|1902|
|     peace|[{Élie, 464, "for...|             null|1902|
|   physics|[{Hendrik A., 2, ...|             null|1902|
|  medicine|[{Ronald, 294, "f...|             null|1902|
|literature|[{Bjørnstjerne, 5...|             null|1903|
| chemistry|[{Svante, 162, "i...|             null|1903|
|     peace|[{Randal, 466, "f...|             null|1903|
|   physics|[{Henri, 4, "in r...|             null|1903|
|  medicine|[{Niels Ryberg, 2..

In [6]:
# 다중 컴럼 정렬 - 연도는 내림차순, 카테고리는 오름차순
df_sorted = df.orderBy(  # 정렬["year", "category"], ascending=[False, True])

# 다중 정렬된 DataFrame 출력
df_sorted.show()  # DataFrame 내용 출력

+----------+--------------------+--------------------+----+
|  category|           laureates|   overallMotivation|year|
+----------+--------------------+--------------------+----+
| chemistry|[{Carolyn, 1015, ...|                null|2022|
| economics|[{Ben, 1021, "for...|                null|2022|
|literature|[{Annie, 1017, "f...|                null|2022|
|  medicine|[{Svante, 1011, "...|                null|2022|
|     peace|[{Ales, 1018, "Th...|                null|2022|
|   physics|[{Alain, 1012, "f...|                null|2022|
| chemistry|[{Benjamin, 1002,...|                null|2021|
| economics|[{David, 1007, "f...|                null|2021|
|literature|[{Abdulrazak, 100...|                null|2021|
|  medicine|[{David, 997, "fo...|                null|2021|
|     peace|[{Maria, 1005, "f...|                null|2021|
|   physics|[{Syukuro, 999, "...|"for groundbreaki...|2021|
| chemistry|[{Emmanuelle, 991...|                null|2020|
| economics|[{Paul, 995, "for...|       

In [7]:
# sort() 함수를 사용한 정렬 - orderBy()와 동일한 기능
df_sorted = df.sort(  # 정렬 (orderBy와 동일)["year", "category"], ascending=[False, True])

# sort() 함수로 정렬된 DataFrame 출력
df_sorted.show()  # DataFrame 내용 출력

+----------+--------------------+--------------------+----+
|  category|           laureates|   overallMotivation|year|
+----------+--------------------+--------------------+----+
| chemistry|[{Carolyn, 1015, ...|                null|2022|
| economics|[{Ben, 1021, "for...|                null|2022|
|literature|[{Annie, 1017, "f...|                null|2022|
|  medicine|[{Svante, 1011, "...|                null|2022|
|     peace|[{Ales, 1018, "Th...|                null|2022|
|   physics|[{Alain, 1012, "f...|                null|2022|
| chemistry|[{Benjamin, 1002,...|                null|2021|
| economics|[{David, 1007, "f...|                null|2021|
|literature|[{Abdulrazak, 100...|                null|2021|
|  medicine|[{David, 997, "fo...|                null|2021|
|     peace|[{Maria, 1005, "f...|                null|2021|
|   physics|[{Syukuro, 999, "...|"for groundbreaki...|2021|
| chemistry|[{Emmanuelle, 991...|                null|2020|
| economics|[{Paul, 995, "for...|       

In [8]:
# 컴럼 이름 변경 - withColumnRenamed() 함수 사용
df_renamed = df.withColumnRenamed(  # 컬럼 이름 변경"category", "Topic")

# 컴럼 이름이 변경된 DataFrame 출력
df_renamed.show()  # DataFrame 내용 출력

+----------+--------------------+--------------------+----+
|     Topic|           laureates|   overallMotivation|year|
+----------+--------------------+--------------------+----+
| chemistry|[{Carolyn, 1015, ...|                null|2022|
| economics|[{Ben, 1021, "for...|                null|2022|
|literature|[{Annie, 1017, "f...|                null|2022|
|     peace|[{Ales, 1018, "Th...|                null|2022|
|   physics|[{Alain, 1012, "f...|                null|2022|
|  medicine|[{Svante, 1011, "...|                null|2022|
| chemistry|[{Benjamin, 1002,...|                null|2021|
| economics|[{David, 1007, "f...|                null|2021|
|literature|[{Abdulrazak, 100...|                null|2021|
|     peace|[{Maria, 1005, "f...|                null|2021|
|   physics|[{Syukuro, 999, "...|"for groundbreaki...|2021|
|  medicine|[{David, 997, "fo...|                null|2021|
| chemistry|[{Emmanuelle, 991...|                null|2020|
| economics|[{Paul, 995, "for...|       

In [9]:
# selectExpr()을 사용한 컴럼 선택 및 이름 변경 - SQL 표현식 사용 가능
df_renamed = (
    df.selectExpr("category as Topic"  # category를 Topic으로 이름 변경
                  , "year as Year"  # year를 Year로 이름 변경
                  , "overallMotivation as Motivation"))  # overallMotivation을 Motivation으로 이름 변경

# selectExpr()로 변경된 DataFrame 출력
df_renamed.show()  # DataFrame 내용 출력

+----------+----+--------------------+
|     Topic|Year|          Motivation|
+----------+----+--------------------+
| chemistry|2022|                null|
| economics|2022|                null|
|literature|2022|                null|
|     peace|2022|                null|
|   physics|2022|                null|
|  medicine|2022|                null|
| chemistry|2021|                null|
| economics|2021|                null|
|literature|2021|                null|
|     peace|2021|                null|
|   physics|2021|"for groundbreaki...|
|  medicine|2021|                null|
| chemistry|2020|                null|
| economics|2020|                null|
|literature|2020|                null|
|     peace|2020|                null|
|   physics|2020|                null|
|  medicine|2020|                null|
| chemistry|2019|                null|
| economics|2019|                null|
+----------+----+--------------------+
only showing top 20 rows



In [10]:
# Spark 세션 종료 - 리소스 정리
spark.stop()  # Spark 세션 종료 - 리소스 정리